# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

In [1]:
include("Mapeos.jl")
include("AutoDiff.jl")

┌ Warning: Deprecated syntax `(u.x′ >= 0) ? print(io, u.x, "+", u.x′, "\u03b5"):` at /home/uzielnmtz/TSFisComp1/tareas/UzielLinares/AutoDiff.jl:18.
│ Use `(u.x′ >= 0) ? print(io, u.x, "+", u.x′, "\u03b5") :` instead.
└ @ nothing /home/uzielnmtz/TSFisComp1/tareas/UzielLinares/AutoDiff.jl:18


Main.AutoDiff

In [2]:
using .Mapeos
using .AutoDiff

In [3]:
function newton_dual(f, guess; n_iter=1000)
    xnn = dual(guess)
    for i in 1:n_iter
        xnn = xnn.x - (f(xnn).x / f(xnn).x′)
        xnn = dual(xnn)
    end
    return xnn.x
end

newton_dual (generic function with 1 method)

In [4]:
Qc(x, c) = x^2 + c

Qc (generic function with 1 method)

In [5]:
fₙ(cn) = (cn[1] - cn[2]) / (cn[2] - cn[3])

fₙ (generic function with 1 method)

In [6]:
function compone(f, x, n)
    n_comp = x
    for i in 1:n
        n_comp = f(n_comp)
    end
    return n_comp
end

compone (generic function with 1 method)

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [7]:
"""
    calcula_cₙ(Fλ, rango_c, n; ϵ=0.01, n_iter=1000)

Calcula el valor aproximado de 'c_n'

"""

function calcula_cₙ(Fλ, rango_c, n; ϵ=0.01, n_iter=1000)
    cn = []
    deltas = []
    ders = []
    x0 = 0.
    for c in rango_c
        F = x -> Fλ(x, c)
        m = Mapeo(F, x0, n_iter)
        atr = dual(m.xnn[end])
        der = compone(F, atr, Int(2^n)).x′
        #println(der)
        δ = abs(der + 1)
        if δ < ϵ
            push!(cn, c)
            push!(ders, der)
            push!(deltas, δ)
        end
        x0 = atr.x
    end
    if !isempty(deltas)
        _, ind = findmin(deltas)
        return cn[ind]
    else
        return Nothing
    end
end

┌ Warning: Deprecated syntax `multiple line breaks between doc string and object`.
│ Use `at most one line break` instead.
└ @ nothing none:8


calcula_cₙ

El anterior codigo busca la bifuracion de periodo $2^n$ en el intervalo `rango_c`, entonces busquemos uno a uno estos puntos.

Por completez, encontremos veamos que estyo encuentra al primer punto de bifuracion de doblamiento de periodo $c_n=-3/4$.

In [41]:
rango_c = reverse(-1.5:1e-2:-1/2.)

-0.5:-0.01:-1.5

In [42]:
c_1 = calcula_cₙ(Qc, rango_c, 0)

-0.75

Continuemos la busqueda tomando intervalos cada vez mas chicos en el cual uno de los extremos es la $c_n$ encontrada previamente.

In [43]:
rango_c = reverse(-1.5:1e-3:c_1)
c_2 = calcula_cₙ(Qc, rango_c, 1)

-1.25

In [44]:
rango_c = reverse(-1.4:1e-4:c_2)
c_3 = calcula_cₙ(Qc, rango_c, 2)

-1.3681

In [45]:
rango_c = reverse(-1.4:1e-4:c_3)
c_4 = calcula_cₙ(Qc, rango_c, 3)

-1.394

In [46]:
rango_c = reverse(-1.5:1e-4:c_4)
c_5 = calcula_cₙ(Qc, rango_c, 4)

-1.3996

In [71]:
rango_c = reverse(-1.5:1e-5:c_5)
c_6 = calcula_cₙ(Qc, rango_c, 5)

-1.40083

In [50]:
rango_c = reverse(-1.5:1e-6:c_6)
c_7 = calcula_cₙ(Qc, rango_c, 6)

-1.446791

In [93]:
# Busquemos este ultimo con mas fineza
rango_c = reverse((c_6-1e-3):1e-8:c_6)
c_7 = calcula_cₙ(Qc, rango_c, 6)

-1.40108525

Entonces, los valores encontrados son los siguientes.

In [94]:
cn = [c_1, c_2, c_3, c_4, c_5, c_6, c_7]

7-element Array{Float64,1}:
 -0.75      
 -1.25      
 -1.3681    
 -1.394     
 -1.3996    
 -1.40083   
 -1.40108525

In [95]:
for (i, c) in enumerate(cn)
    println(i, "  ", c)
end

1  -0.75
2  -1.25
3  -1.3681
4  -1.394
5  -1.3996
6  -1.40083
7  -1.40108525


In [ ]:
for (i in cn[3:end]
    

In [96]:
cn[end-3:end]

4-element Array{Float64,1}:
 -1.394     
 -1.3996    
 -1.40083   
 -1.40108525

In [98]:
fₙ(cn[end-3:end-1])

4.552845528455086

In [84]:
(cn[6] - cn[5]) / (cn[7] - cn[6])

4.818805093047623

## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

In [77]:
Sc(x, c) = c * sin(x)

Sc (generic function with 1 method)

Repitamos el mismo procedimiento que el empleado para el mapeo anterior.

In [112]:
rango_c = 0:0.5:10
c_1 = calcula_cₙ(Sc, rango_c, 0)

Nothing

In [117]:
Mapeo(x -> Sc(x, 0.1), 0.1, 1000)

Mapeo(getfield(Main, Symbol("##10#11"))(), 0.1, 1000, [0.1, 0.00998334, 0.000998318, 9.98317e-5, 9.98317e-6, 9.98317e-7, 9.98317e-8, 9.98317e-9, 9.98317e-10, 9.98317e-11  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.1, 0.1, 0.00998334, 0.00998334, 0.000998318, 0.000998318, 9.98317e-5, 9.98317e-5, 9.98317e-6, 9.98317e-6  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-10000.0, 0.00998334, 0.00998334, 0.000998318, 0.000998318, 9.98317e-5, 9.98317e-5, 9.98317e-6, 9.98317e-6, 9.98317e-7  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], false)

In [114]:
Sc(dual(0), 0)

0.0+0.0ε

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.

In [ ]:
x0 = 0.

In [129]:
Qc(0, Qc(0, 3))

3

In [125]:
compone(c -> Qc(0, c), 2, 4)

2

In [120]:
newton_dual(c -> Qc(0, c), 2.)

0.0